# Throughtput Benchmarking  Seldon-Core on GCP Kubernetes

The notebook will provide a benchmarking of seldon-core for maximum throughput test. We will run a stub model and test using REST and gRPC predictions. This will provide a maximum theoretical throughtput for model deployment in the given infrastructure scenario:
  
   * 1 replica of the model running on n1-standard-16 GCP node
   
For a real model the throughput would be less. Future benchmarks will test realistic models scenarios.


## Create Cluster

Create a cluster of 4 nodes of machine type n1-standard-16. You can use GKE console or `gcloud` command line.

## Cordon off loadtest nodes

In [1]:
!kubectl get nodes

NAME                                                STATUS   ROLES    AGE   VERSION
gke-standard-cluster-8-default-pool-415f5c23-gpnp   Ready    <none>   30s   v1.13.11-gke.14
gke-standard-cluster-8-default-pool-415f5c23-lzdf   Ready    <none>   30s   v1.13.11-gke.14
gke-standard-cluster-8-default-pool-415f5c23-v6sr   Ready    <none>   30s   v1.13.11-gke.14
gke-standard-cluster-8-default-pool-415f5c23-z9g6   Ready    <none>   30s   v1.13.11-gke.14


We cordon off first 3 nodes so seldon-core and the model will not be deployed on the 1 remaining node.

In [2]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node/gke-standard-cluster-8-default-pool-415f5c23-gpnp cordoned
node/gke-standard-cluster-8-default-pool-415f5c23-lzdf cordoned
node/gke-standard-cluster-8-default-pool-415f5c23-v6sr cordoned


Label the nodes so they can be used by locust.

In [3]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}') role=locust

node/gke-standard-cluster-8-default-pool-415f5c23-gpnp labeled
node/gke-standard-cluster-8-default-pool-415f5c23-lzdf labeled
node/gke-standard-cluster-8-default-pool-415f5c23-v6sr labeled


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

## Create Stub Deployment

In [4]:
!pygmentize resources/loadtest_simple_model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-core-loadtest"
    },
    "spec": {
        "annotations": {
            "project_name": "loadtest",
            "deployment_version": "v1"
        },
        "name": "loadtest",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                        ],
                        "terminationGracePeriodSeconds": 20
                    }
                }],
                "graph": {
                    "children": [],
                    "name": "stub",
                    "endpoint": {
			"type" : "REST"
		    },
                    "type": "MODEL",
		    "implementation": "SIMPLE_MODEL"
                },
                "name": "l

In [5]:
!kubectl apply -f resources/loadtest_simple_model.json

seldondeployment.machinelearning.seldon.io/seldon-core-loadtest created


Wait for deployment to be running.

In [7]:
!kubectl rollout status deployment.apps/loadtest-loadtest-9eecb7d

Error from server (NotFound): deployments.apps "loadtest-loadtest-9eecb7d" not found


## Run benchmark

Uncorden the first 3 nodes so they can be used to schedule locust

In [12]:
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node/gke-standard-cluster-4-default-pool-06d14028-76f3 uncordoned
node/gke-standard-cluster-4-default-pool-06d14028-cx68 uncordoned
node/gke-standard-cluster-4-default-pool-06d14028-mvdn uncordoned


## gRPC
Start locust load test for gRPC

In [13]:
!helm install loadtest ../helm-charts/seldon-core-loadtesting \
    --set locust.host=loadtest-seldon-core-loadtest:5001 \
    --set locust.script=predict_grpc_locust.py \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=64 

NAME: loadtest
LAST DEPLOYED: Mon Dec  2 10:32:54 2019
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


To download stats use 

```bash
if [ "$#" -ne 2 ]; then
    echo "Illegal number of parameters: <experiment> <rest|grpc>"
fi

EXPERIMENT=$1
TYPE=$2

MASTER=`kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}'`

kubectl cp ${MASTER}:stats_distribution.csv ${EXPERIMENT}_${TYPE}_stats_distribution.csv
kubectl cp ${MASTER}:stats_requests.csv ${EXPERIMENT}_${TYPE}_stats_requests.csv
```

You can get live stats by viewing the logs of the locust master

In [20]:
!kubectl logs $(kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}') --tail=10

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s
--------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                              0     0(0.00%)                                       0.00

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s
--------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                   

In [21]:
!helm delete loadtest

release "loadtest" uninstalled


## REST 
Run REST benchmark

In [22]:
!helm install loadtest ../helm-charts/seldon-core-loadtesting \
    --set locust.host=http://loadtest-seldon-core-loadtest:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=64

NAME: loadtest
LAST DEPLOYED: Mon Dec  2 10:37:24 2019
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


Get stats as per gRPC and/or monitor

In [33]:
!kubectl logs $(kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}') --tail=10

 POST predictions                                              208902 208902(50.00%)      12       4      74  |      11 4471.40
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                         208902 208902(100.00%)                                    4471.40

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s
--------------------------------------------------------------------------------------------------------------------------------------------
 POST predictions                                              218526 218526(50.00%)      12       4      74  |      11 4463.30
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                         21

In [34]:
!helm delete loadtest

release "loadtest" uninstalled


In [35]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node/gke-standard-cluster-4-default-pool-06d14028-76f3 cordoned
node/gke-standard-cluster-4-default-pool-06d14028-cx68 cordoned
node/gke-standard-cluster-4-default-pool-06d14028-mvdn cordoned


## Tear Down

In [36]:
!kubectl delete -f resources/loadtest_simple_model.json

seldondeployment.machinelearning.seldon.io "seldon-core-loadtest" deleted


In [38]:
!helm delete seldon-core

Error: uninstall: Release not loaded: seldon-core: release: not found
